In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [2]:
import pickle
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout, LSTM
from keras.optimizers import Adam
import numpy as np
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
data_folder = '/content/gdrive/MyDrive/University/master/term1/Machine-Learning/ML-Project/Data/mfcc/'

In [ ]:
SHAPE_SIZE = (441022,1)

In [ ]:
file_name  = data_folder + "audio_segment.pkl"
file_name_labels  = data_folder + "audio_segment_labels.pkl"
with open(file_name, 'rb') as file:
    uncleaned_data = pickle.load(file)

with open(file_name_labels, 'rb') as file:
    uncleaned_labels = pickle.load(file)
np.shape(uncleaned_data), np.shape(uncleaned_labels)

/usr/local/lib/python3.8/dist-packages/numpy/core/fromnumeric.py:1970: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


((9538,), (9538,))

In [ ]:
np.shape(uncleaned_data[0])

(441023,)

In [ ]:
data = []
labels = []
for k, j in zip(uncleaned_data,uncleaned_labels):
    k = np.expand_dims(k, axis=1)
    if np.shape(k) != SHAPE_SIZE:
        print(np.shape(k))
    else:
        data.append(k)
        labels.append(j)
labels = np.array(labels)
data = np.array(data)
uncleaned_data = []
uncleaned_labels = []
np.shape(data), np.shape(labels)

(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(440339, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(440890, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(440714, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441023, 1)
(441

((8914, 441022, 1), (8914,))

In [ ]:
np.unique(labels)

array([0, 1, 2, 3, 4, 5, 6])

In [ ]:
import numpy
a = numpy.array(labels)
unique, counts = numpy.unique(a, return_counts=True)
dict(zip(unique, counts))

{0: 1171, 1: 1184, 2: 1331, 3: 1367, 4: 1142, 5: 1420, 6: 1299}

In [ ]:
labels = to_categorical(labels)
x_train, x_test, y_train, y_test =  train_test_split(data, labels, test_size=0.2, shuffle=True, random_state=42, stratify=labels)
data = []
labels = []

In [ ]:
np.shape(x_train), np.shape(y_train)

((7131, 441022, 1), (7131, 7))

In [ ]:
class Trainer:
    def __init__(self, x_train, y_train, number_of_class=7 ,input_size=None, epochs=1000, optimizer='adam'):
        self.input_size = input_size
        self.model = Sequential()
        self.x_train, self.y_train = x_train, y_train
        self.epochs = epochs
        self.optimizer = optimizer
        self.number_of_class = number_of_class
        self.callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=50)
    # define cnn model
    def define_model(self):
        print("Build LSTM RNN model ...")
        self.model = Sequential()

        self.model.add(LSTM(units=128, dropout=0.5, return_sequences=True, input_shape=self.input_size))
        self.model.add(LSTM(units=32,  dropout=0.5, return_sequences=False))
        self.model.add(Dense(units=self.number_of_class, activation="softmax"))
        self.model.compile(optimizer=self.optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
        return self.model.summary()

    def fit_model(self):
        # fit model
        self.history = self.model.fit(self.x_train, self.y_train, epochs=self.epochs, batch_size=128,
                                      validation_split=0.2, verbose=1,shuffle=True, 
                                      callbacks=[self.callback])

    def plot_history(self):
        fig = plt.figure(figsize=(12, 4))
        metrics = ['loss', 'accuracy']
        for n, metric in enumerate(metrics):
            plt.subplot(1, 2, n + 1)
            plt.plot(self.history.epoch, self.history.history[metric], label='Train')
            plt.plot(self.history.epoch, self.history.history[f"val_{metric}"], linestyle="--", label='Validation')
            plt.xlabel('Epoch')
            plt.ylabel(metric)
            plt.title(metric)
        plt.legend()
        plt.show()

    def evaluate(self, x_test, y_test):
        y_pred = np.argmax(self.model.predict(x_test), axis=-1)
        print(classification_report(np.argmax(y_test, axis=-1), y_pred, digits=3))



In [ ]:
trainer = Trainer(x_train, y_train, input_size = SHAPE_SIZE, epochs=1000)
trainer.define_model()

Build LSTM RNN model ...
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 441022, 128)       66560     
                                                                 
 lstm_1 (LSTM)               (None, 32)                20608     
                                                                 
 dense (Dense)               (None, 7)                 231       
                                                                 
Total params: 87,399
Trainable params: 87,399
Non-trainable params: 0
_________________________________________________________________


In [ ]:
trainer.fit_model()

Epoch 1/1000


ResourceExhaustedError: ignored

In [ ]:
trainer.plot_history()

In [ ]:
trainer.evaluate(x_test, y_test)

In [ ]:
trainer.evaluate(x_train, y_train)